In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
import requests
import numpy as np
import re
from matplotlib import pyplot as plt
import winsound, time

In [2]:
def beepme():
    duration = 1000  # milliseconds
    freq = 440  # Hz
    while(True):
        winsound.Beep(freq, duration)
        time.sleep(duration*2)

In [3]:
sub = {}
num = {}
pre = {}

for quarter in os.listdir('./data/edgar-download/'):
#     if bool(re.search(r'20[0-2][0-57-9]q[1-4]',quarter)):
#         print(quarter+' skipped')
#         continue
#     print(quarter)

    sub[quarter] = pd.read_csv(r'./data/edgar-download/' + quarter + r'/sub.txt', delimiter= '\t', dtype='str')
    num[quarter] = pd.read_csv(r'./data/edgar-download/' + quarter + r'/num.txt', delimiter= '\t', encoding='unicode_escape', dtype='str')
    pre[quarter] = pd.read_csv(r'./data/edgar-download/' + quarter + r'/pre.txt', delimiter= '\t', encoding='unicode_escape', error_bad_lines=False, dtype='str')

print('Complete')

b'Skipping line 620821: expected 10 fields, saw 11\n'
b'Skipping line 884837: expected 10 fields, saw 11\n'


Complete


In [461]:
winsound.Beep(440,5000)

In [6]:
CIK = pd.DataFrame()

for quarter in sub.keys():
    t = pd.Series(index = sub[quarter]['cik'].drop_duplicates(), name=quarter)
# t = pd.Series(index= sub['2009q4']['cik'], name='2009q3')
    t[:]=True
    CIK = CIK.join(t, how='outer')
    quarters = CIK.filter(regex=r'20[1-2][2-9]').columns
    
# Filter CIKs by those who have complete datasets from 2012 onwards
CIK = CIK[CIK.filter(regex=r'20[1-2][2-9]').transpose().count() == 32].filter(regex=r'20[1-2][2-9]').index.values



In [460]:
# Dicts to store most common (of their respective name) for each quarter
TAGS = {} 
ADSH = {} 
NUMS = {}
PIVOT = {}

for quarter in quarters:
    
    print(quarter)

    
    # Choose ADSH from 'sub' sheet, filter CIKs from master list, and form from the most popular form of the quarter
    ADSH[quarter] = sub[quarter][
        (sub[quarter]['form'] == sub[quarter]['form'].value_counts().index[sub[quarter]['form'].value_counts().index.str.match(r'10-')][0])
        & (sub[quarter]['cik'].isin(CIK))
    ].sort_values(by='accepted').drop_duplicates(subset=['cik','form'], keep='last')[['cik','adsh']].sort_values(by='cik',ascending=False)
    
    ADSH[quarter] = ADSH[quarter]['cik'].str.pad(width=10,side='left',fillchar='0').str.cat(ADSH[quarter]['adsh'].str.split('-').str[1], sep='-').str.cat(ADSH[quarter]['adsh'].str.split('-').str[2], sep='-')
    
    pre[quarter].columns = pre[quarter].columns.str.strip() # strip text, not all formatted correctly
    
    
    TAGS[quarter] = pre[quarter][
        (pre[quarter]['adsh'].isin(ADSH[quarter])) # Choose ADSH from chosen list
        & (pre[quarter]['stmt'] == 'BS')  # Choose the balance sheet
    ]['tag'].value_counts()[:25]

    NUMS[quarter] = num[quarter][
        (num[quarter]['adsh'].isin(ADSH[quarter])) 
        & (num[quarter]['tag'].isin(TAGS[quarter].index))
    ][['adsh','tag','value']]
    
    NUMS[quarter] = NUMS[quarter].drop_duplicates(subset=['adsh','tag'], keep='first').sort_index()

    # PIVOT the table so that the rows are the ADSH, and columns are the common tags
    PIVOT[quarter] = NUMS[quarter].pivot(index='adsh', columns='tag')
    PIVOT[quarter].columns = PIVOT[quarter].columns.droplevel()
    PIVOT[quarter] = PIVOT[quarter].sort_index()

    
a = pd.DataFrame(TAGS[quarters[0]]).index
b = pd.Series(index= NUMS[quarters[0]]['adsh'].str.split('-').str[0]).index.drop_duplicates()

for quarter in quarters:
    a = a.join(TAGS[quarter].index,how='inner')
    c = pd.Series(index= NUMS[quarter]['adsh'].str.split('-').str[0]).index.drop_duplicates()
    b = b.join(c, how='inner')
    
    
for quarter in quarters:
    PIVOT[quarter] = PIVOT[quarter].loc[:, PIVOT[quarter].columns.isin(a)]
    PIVOT[quarter] = PIVOT[quarter].reindex(sorted(PIVOT[quarter].columns), axis=1)
    PIVOT[quarter] = PIVOT[quarter][PIVOT[quarter].index.str.split('-').str[0].isin(b)].sort_index()

2012q1
2012q2
2012q3
2012q4
2013q1
2013q2
2013q3
2013q4
2014q1
2014q2
2014q3
2014q4
2015q1
2015q2
2015q3
2015q4
2016q1
2016q2
2016q3
2016q4
2017q1
2017q2
2017q3
2017q4
2018q1
2018q2
2018q3
2018q4
2019q1
2019q2
2019q3
2019q4


In [468]:
PIVOT[q].to_dict('index')

{'0000002178-19-000107': {'AccumulatedOtherComprehensiveIncomeLossNetOfTax': nan,
  'Assets': '313845000.0000',
  'AssetsCurrent': '236253000.0000',
  'CashAndCashEquivalentsAtCarryingValue': '117066000.0000',
  'Goodwill': nan,
  'LiabilitiesCurrent': '123267000.0000',
  'OtherAssetsNoncurrent': nan,
  'PropertyPlantAndEquipmentNet': '44623000.0000',
  'RetainedEarningsAccumulatedDeficit': '134228000.0000',
  'StockholdersEquity': '147119000.0000'},
 '0000004904-19-000050': {'AccumulatedOtherComprehensiveIncomeLossNetOfTax': '0.0000',
  'Assets': '8394100000.0000',
  'AssetsCurrent': '373100000.0000',
  'CashAndCashEquivalentsAtCarryingValue': '24500000.0000',
  'Goodwill': '52500000.0000',
  'LiabilitiesCurrent': '710200000.0000',
  'OtherAssetsNoncurrent': '2794800000.0000',
  'PropertyPlantAndEquipmentNet': '10992100000.0000',
  'RetainedEarningsAccumulatedDeficit': '10095300000.0000',
  'StockholdersEquity': '2422500000.0000'},
 '0000005513-19-000133': {'AccumulatedOtherComprehens